In [1]:
## Auto AI pieces
# - Profiler: https://www.geeksforgeeks.org/data-profiling-in-pandas-using-python/
# - AutoAI: https://www.kdnuggets.com/2019/01/automated-machine-learning-python.html
# - lale: https://nbviewer.jupyter.org/github/IBM/lale/blob/master/examples/docs_guide_for_sklearn_users.ipynb
# - auto-sklearn: https://automl.github.io/auto-sklearn/master/examples/20_basic/example_classification.html#sphx-glr-examples-20-basic-example-classification-py

## Data Profiling

In [2]:
## Install as needed
#!pip install -U pandas-profiling

In [3]:
import pandas as pd 
import pandas_profiling 

In [4]:
# Local data cache
datadir = '../common-data/covid/'
datafile = datadir + "data.csv"

In [5]:
# Read data from local file
data = pd.read_csv(datafile, parse_dates=['date'])
data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [6]:
# Run the profile report 
profile = data.profile_report(title='Pandas Profiling Report') 

In [7]:
# Saving the report as html file 
profile.to_file(output_file="data/coviddata_profiling.html") 

Summarize dataset:   0%|          | 0/19 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Lale - select method and tune paramaters

In [8]:
## Install if needed
#! pip install lale

In [9]:
# Import the datasets
import lale.datasets

In [10]:
# Load a data and see inside
(train_X, train_y), (test_X, test_y) = lale.datasets.california_housing_df()
data = pd.concat([train_X.head(), train_y.head()], axis=1)

In [11]:
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,3.34502,33.000000,5.081655,1.040112,1364.200000,2.889498,34.122000,-118.516000,1.695200
std,0.85071,17.363755,0.903396,0.057128,575.154936,0.944104,1.697254,1.566662,1.232384
min,1.94250,4.000000,4.002817,0.985119,874.000000,1.738095,32.690000,-120.480000,0.934000
25%,3.25960,33.000000,4.473545,1.006421,915.000000,2.300000,32.710000,-119.800000,0.965000
50%,3.55420,36.000000,5.017657,1.033803,1314.000000,2.723214,33.770000,-118.160000,1.030000
75%,3.81250,43.000000,5.645833,1.041005,1418.000000,3.691814,34.660000,-117.110000,1.726000
max,4.15630,49.000000,6.268421,1.134211,2300.000000,3.994366,36.780000,-117.030000,3.821000


In [12]:
import numpy as np

import sklearn.metrics
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor as Tree
from sklearn.pipeline import Pipeline

from lale.lib.lale import Hyperopt

/Users/biplavs/opt/anaconda3/lib/python3.7/site-packages/pyparsing.py:3174: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [13]:
lale.wrap_imported_operators()

INFO:lale.operator_wrapper:Lale:Wrapped known operator:PCA
INFO:lale.operator_wrapper:Lale:Wrapped known operator:Tree
INFO:lale.operator_wrapper:Lale:Wrapped known operator:Pipeline


In [14]:
pca_tree_planned = Pipeline(steps=[('tfm', PCA), ('estim', Tree)])

In [15]:
%%time
pca_tree_trained = pca_tree_planned.auto_configure(
    train_X, train_y, optimizer=Hyperopt, cv=3, max_evals=10, verbose=True)

INFO:lale.schema_simplifier:simplifyAny: skipping not for optimizer {'description': 'Number of components to keep.', 'type': 'integer', 'minimum': 1, 'maximum': 8, 'forOptimizer': False} (after simplification)
INFO:lale.schema_simplifier:simplifyAll: skipping not for optimizer {'description': 'Number of components to keep.', 'type': 'integer', 'minimum': 1, 'maximum': 8, 'forOptimizer': False} (after simplification)
INFO:lale.schema_simplifier:mergeAll: conflicting description fields: This class does not support sparse input. See TruncatedSVD for an alternative with sparse data. and This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.
 found when merging schemas [{'description': 'This class does not support sparse input. See TruncatedSVD for an alternative with sparse data.', 'type': 'object'}, {'description': 'This first object lists all constructor arguments with their types, but omits constraints for conditional h

INFO:lale.schema_simplifier:mergeAll: conflicting description fields: This class does not support sparse input. See TruncatedSVD for an alternative with sparse data. and This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.
 found when merging schemas [{'description': 'This class does not support sparse input. See TruncatedSVD for an alternative with sparse data.', 'type': 'object'}, {'description': 'This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.\n', 'type': 'object', 'properties': {'n_components': {'enum': [None, 'mle']}, 'whiten': {'default': False, 'description': 'When true, multiply the components vectors by the square root of\nn_samples and then divide by the singular values to ensure uncorrelated\noutputs with unit component-wise variances.', 'type': 'boolean'}, 'svd_solver': {'default': 'auto', 'description': 'Algorithm to use.', 'enum'

INFO:lale.schema_simplifier:mergeAll: conflicting description fields: This class does not support sparse input. See TruncatedSVD for an alternative with sparse data. and This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.
 found when merging schemas [{'description': 'This class does not support sparse input. See TruncatedSVD for an alternative with sparse data.', 'type': 'object'}, {'description': 'This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.\n', 'type': 'object', 'properties': {'n_components': {'enum': [None, 'mle']}, 'whiten': {'default': False, 'description': 'When true, multiply the components vectors by the square root of\nn_samples and then divide by the singular values to ensure uncorrelated\noutputs with unit component-wise variances.', 'type': 'boolean'}, 'svd_solver': {'default': 'auto', 'description': 'Algorithm to use.', 'enum'

INFO:lale.schema_simplifier:simplifyAll: [{'default': 'auto', 'description': 'Algorithm to use.', 'enum': ['auto', 'full', 'arpack', 'randomized']}, {'enum': ['full', 'auto']}, {'enum': ['arpack']}, {'enum': ['randomized']}] is not a satisfiable list of conjoined schemas because the enumeration ['auto', 'full', 'arpack', 'randomized'] has no elements that are satisfiable by the conjoined schemas
INFO:lale.schema_simplifier:simplifyAll: required key svd_solver is False, so the entire conjugation of schemas [{'description': 'This class does not support sparse input. See TruncatedSVD for an alternative with sparse data.', 'type': 'object'}, {'description': 'This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.\n', 'type': 'object', 'properties': {'n_components': {'enum': [None, 'mle']}, 'whiten': {'default': False, 'description': 'When true, multiply the components vectors by the square root of\nn_samples and then divide

INFO:lale.schema_simplifier:mergeAll: conflicting description fields: This class does not support sparse input. See TruncatedSVD for an alternative with sparse data. and This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.
 found when merging schemas [{'description': 'This class does not support sparse input. See TruncatedSVD for an alternative with sparse data.', 'type': 'object'}, {'description': 'This first object lists all constructor arguments with their types, but omits constraints for conditional hyperparameters.\n', 'type': 'object', 'properties': {'n_components': {'enum': [None, 'mle']}, 'whiten': {'default': False, 'description': 'When true, multiply the components vectors by the square root of\nn_samples and then divide by the singular values to ensure uncorrelated\noutputs with unit component-wise variances.', 'type': 'boolean'}, 'svd_solver': {'default': 'auto', 'description': 'Algorithm to use.', 'enum'

...

 80%|████████  | 8/10 [00:03<00:00,  2.43trial/s, best loss: -0.41410769000479447]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.018215 seconds
INFO:hyperopt.tpe:TPE using 8/8 trials with best loss -0.414108


 90%|█████████ | 9/10 [00:03<00:00,  2.34trial/s, best loss: -0.41410769000479447]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.019009 seconds
INFO:hyperopt.tpe:TPE using 9/9 trials with best loss -0.414108


100%|██████████| 10/10 [00:04<00:00,  2.34trial/s, best loss: -0.41410769000479447]
CPU times: user 44.5 s, sys: 10.3 s, total: 54.7 s
Wall time: 5.72 s


In [16]:
# Predictions
predicted = pca_tree_trained.predict(test_X)
print(f'R2 score {sklearn.metrics.r2_score(test_y, predicted):.2f}')


R2 score 0.37


## Automated Algo Selection

In [17]:
## Install and other issues
# !pip install auto-sklearn
## Note: If you face issue with swig, install with brew install swig

In [18]:
#!pip install -U scikit-learn
#!pip install auto-sklearn

In [19]:
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

In [20]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

In [21]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='data/autosklearn_classification_example_tmp',
    output_folder='data/autosklearn_classification_example_out',
)
automl.fit(X_train, y_train, dataset_name='breast_cancer')

AutoSklearnClassifier(output_folder='data/autosklearn_classification_example_out',
                      per_run_time_limit=30, time_left_for_this_task=120,
                      tmp_folder='data/autosklearn_classification_example_tmp')

In [22]:
# Print the model
print(automl.show_models())

[(0.160000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'passive_aggressive', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:passive_aggressive:C': 0.14268277711454813, 'classifier:passive_aggressive:average': 'False', 'classifier:passive_aggressive:fit_intercept': 'True', 'classifier:passive_aggressive:loss': 'hinge', 'classifier:passive_aggressive:tol': 0.0002600768160857831, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.0070580904199417415},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multi

In [23]:
# Performance metric
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.9440559440559441
